In [1]:
"""Sub-Holding Keys

Demonstration of how to set up and use sub-holding keys

Attributes
----------
properties
sub-holding keys
cocoon - seed_data
holdings
prorated
"""

'Sub-Holding Keys\n\nDemonstration of how to set up and use sub-holding keys\n\nAttributes\n----------\nproperties\nsub-holding keys\ncocoon - seed_data\nholdings\nprorated\n'

In [2]:
# %load_ext lab_black
# %load_ext nb_black

# 1. Sub-Holding Keys

This notebook demonstrates LUSID's [Sub-holding Keys](https://support.finbourne.com/what-are-subholding-keys) (or SHKs). The core idea with `Sub-holding Keys` - they allow you to bucket your `holding` in one instrument (or [LUID](https://support.finbourne.com/what-is-a-lusid-unique-identifier-luid)) into different groups. For example, in this notebook we have a `Sub-Holding Key` of <i>strategy</i> which is used to tag transactions on the same instrument following different investment strategies. Then in the `holdings` report you can see the position split-out into two buckets. However, this is just one sample implementation of `Sub-Holding Keys`. You are allowed use <u>any</u> pre-defined transaction property as a `Sub-Holding Key`. 

# 2. Setup LUSID

In [3]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta
import pytz

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.exceptions import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

Load a mapping file for DataFrame headers for the `build transaction` and `get holdings` response.

In [4]:
with open(r"config/build_transactions_mapping.json") as mappings_file:
    build_transactions_json_mapping = json.load(mappings_file)

with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

Define our transaction portfolios API

In [5]:
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

# 3. Load Data

## 3.1 Declare a scope and load our CSV file

In [6]:
# Create a new scope

scope = "notebook_shk1"
portfolio_code = "EQUITY_UK" + "_" + create_scope_id().replace("-", "")

In [7]:
# Load a file of equity transactions

transactions_file = r"data/shk/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code
transactions_df.tail(2)

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
20,EQUITY_UK_399ef2a74950ea,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,cash_001,FundsIn,02/01/2020,04/01/2020,12000000,1,12000000,GBP,ftse_tracker,GBP
21,EQUITY_UK_399ef2a74950ea,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,cash_002,FundsIn,02/02/2020,04/01/2020,100000,1,100000,GBP,food_retail,GBP


## 3.2 Create a property for the new Sub-Holding key

The <b>strategy</b> will be used to create our `Sub-Holding Key` on the portfolio.

In [8]:
domain = "Transaction"
scope = scope
prop_code = "strategy"

try:
    api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
        create_property_definition_request=lusid.models.CreatePropertyDefinitionRequest(
            domain=domain,
            scope=scope,
            code=prop_code,
            value_required=None,
            display_name="Investment strategy",
            data_type_id=lusid.ResourceId(scope="system", code="string"),
            life_time=None,
        )
    )

except lusid.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])

## 3.3 Load default transactions into a new scope

The portfolio is created with the new `Sub-holding Key`.

In [9]:
# Load portfolios, instruments, and transactions

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_df,
    "DataFrame",
    sub_holding_keys=[f"Transaction/{scope}/strategy"],
)

# 4. Lets check our holdings

We have can see that the one Tesco instrument (with the same LUID) is bucketed under two different `Sub-Holding Keys` with the <b>strategy</b> label. There is also a seperate CCY_GBP cash line for tracking the cash in each SHK.

In [10]:
response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name", "Instrument/default/ClientInternal"],
    filter="properties.Instrument/default/Name in ('Tesco', 'CCY_GBP')",
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df

,LusidInstrumentId,strategy(notebook_shk1-SubHoldingKeys),InstrumentName,ClientInternal(default-Properties),SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy,currency
0,LUID_447EXYXY,ftse_tracker,Tesco,EQ_1240,EQUITY_UK_399ef2a74950ea,notebook_shk1,P,8000.0,8000.0,64000.0,GBP,0.0,GBP,GBP
1,LUID_447EXYXY,food_retail,Tesco,EQ_1240,EQUITY_UK_399ef2a74950ea,notebook_shk1,P,4000.0,4000.0,36000.0,GBP,0.0,GBP,GBP


# 5. Book a proated transaction across two SHKs

In this section we book one transaction and do a prorated allocation across two SHKs.

## 5.1 Create a new transaction type

First we create a new "Buy" transaction with the prorata configuration. Specifically we add a `AllocationMethod=Prorated` property.

In [11]:
# Prepare the Transaction Type model for new transaction type

new_transaction_config = [
    models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type="BuyProRated",
                description="An BuyProRated transaction type",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties={},
                mappings=[],
            ),
            models.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="Side2",
                direction=-1,
                properties={},
                mappings=[],
            ),
        ],
        properties={"TransactionConfiguration/default/AllocationMethod": models.PerpetualProperty(
                    key="TransactionConfiguration/default/AllocationMethod",
                value=models.PropertyValue(label_value="Prorated"))}
    )
]

In [12]:
# Upload the transaction type

new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)

## 5.2 Book a transaction

Next we book a Tesco PLC transaction for 6000 units. We already hold Tesco PLC in the portfolio, so we expect the new transaction to be alloacted as follows:

1. The Movements Engine allocates 4000 units to the <b>ftse_tracker</b> strategy
2. The Movements Engine allocates 2000 units to the <b>food_retail</b> strategy

In [13]:
request = transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id="trd_0021PRORATE",
            type="BuyProRated",
            instrument_identifiers={"Instrument/default/ClientInternal": "EQ_1240"},
            transaction_date="2020-01-29",
            settlement_date="2020-01-31",
            units=6000,
            transaction_price=models.TransactionPrice(price=10),
            total_consideration=models.CurrencyAndAmount(amount=60000, currency="GBP"),
            exchange_rate=None,
            counterparty_id=None,
            source="",
            properties={},
        )
    ],
)

Then check the results:

In [14]:
response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name", "Instrument/default/ClientInternal"],
    filter="properties.Instrument/default/Name in ('Tesco', 'CCY_GBP')",
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df

,LusidInstrumentId,strategy(notebook_shk1-SubHoldingKeys),InstrumentName,ClientInternal(default-Properties),SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy,currency
0,LUID_447EXYXY,ftse_tracker,Tesco,EQ_1240,EQUITY_UK_399ef2a74950ea,notebook_shk1,P,12000.0,12000.0,104000.0,GBP,0.0,GBP,GBP
1,LUID_447EXYXY,food_retail,Tesco,EQ_1240,EQUITY_UK_399ef2a74950ea,notebook_shk1,P,6000.0,6000.0,56000.0,GBP,0.0,GBP,GBP
